#  Analysis of  electronics recycling company "X".

## Problem overview.

In 2003 the European Community establised the WEEE (Waste Electrical and Electronics Equipment) directive to set collection, recycling and recovery targets for all types of electrical goods. 

If you distribute electrical and/or electronic equipment you must comply with WEEE directives or face fines. Distribution is not limited to vendors only if you hand out equipment to your employees you must adhere to WEEE regulations, for individual users responsible disposal is encouraged. 

Instead of dealing with the process themselves, companies subcontract the recycling process to a third party which for a fee will guarantee the collection and recycling process is performed correctly and fully documented. Several companies offer this type of service, company "X" is one of them.

To expand its business portfolio company "X" also resells/refurbish/remanufacture the equipment received. However, in this case company "X" buys the equipment from companies and individual users hoping to recover as much as possible from the equipment. 

Several factors affect the return on investment (e.g.  Initial buying price, type of customer,  product type) and company "X" is keen on finding out how to maximise it.  Company "X" has been  recording the product mix,  the customers and  buyers they deal with and their business processes for the  20 months.  They want to use data science to understand  what to buy, when  to buy, who to buy it from and how to process  their product  to maximise their  return on investment.

##  Composition of the product  mix.

In [25]:
## Import required libraries.
%matplotlib inline
import matplotlib as mtl
import seaborn as sns
mtl.rcParams['savefig.dpi'] = 144
import mysql.connector
import os
import pandas as pd
import numpy as np
import re
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import column
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from datetime import datetime, timedelta
from IPython.display import HTML

<img src='Images/environment-WEEE-logo.png'>

* If you distribute electrical and/or electronic equipement you are responsible for disposing in a safe and traceable manner.
* Company "X" offers a traceable disposal service. Company "X" also purchases used equipement to recover as much of it as possible.
* Company "X" has been recording data on their products for the last 20 months, they want to know what to buy, when to buy, who to buy it from and how to process their product to maximise their return on investment.

In [26]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [27]:
## Read the server credentials from settings file.
filePath = os.path.join(os.getcwd(), 'Settings')
with open(os.path.join(filePath,'settingsSQL.txt'), 'r') as settingsFile:
    content = settingsFile.readlines()
content = [item.strip().split(' = ')[1] for item in content]

In [28]:
## Connection to the database.
conn = mysql.connector.connect(host = content[0],
                               user = content[1],
                               password = content[2],
                               db= content[3])
cursor = conn.cursor()

In [29]:
## Read the ExchangeRate-API credentials from settings file.
filePath = os.path.join(os.getcwd(), 'Settings')
with open(os.path.join(filePath,'settingsExchangeRate.txt'), 'r') as settingsFile:
    content = settingsFile.readlines()
content = [item.strip().split(' = ')[1] for item in content]
exchangeRateAPICredentials = content[0]

In [30]:
## Product mix per month.
cursor.execute(r"SELECT product_type.type AS prod_type, DATE(DATE_FORMAT(process_history.start_time, '%Y-%m-01')) AS registration_month, \
                COUNT(product_type.type) AS product_per_month \
                FROM product \
                INNER JOIN product_type ON product.product_type_idproduct_type = product_type.idproduct_type \
                INNER JOIN process_history ON product.idproduct = process_history.product_idproduct \
                GROUP BY prod_type, registration_month");
rows = cursor.fetchall()
connectionClosed = cursor.close()

In [31]:
## Construct a dataframe with the results.
df = pd.DataFrame(list(rows), columns = ['productType', 'date', 'numberPerMonth'])

In [32]:
## Remove data with None in the date column.
noneDFData = df[df['date'].isnull()]
df = df.dropna(axis=0, how='any')
rawTable = df.copy()
rawTable['year'] = rawTable['date'].apply(lambda x:x.year)
rawTable['month'] = rawTable['date'].apply(lambda x:x.month)

## Ratio of products per year. --IMPORTANT: Only the month_year column is set as timestamp format.--
tempGroupedSeries = rawTable.groupby(['year']).sum()['numberPerMonth']
tempGroupedDataFrame = rawTable.groupby(['productType', 'year']).sum()[['numberPerMonth']]
tempGroupedDataFrame = tempGroupedDataFrame.rename(columns={'numberPerMonth':'totalsPerYearPerProduct'})
tempGroupedDataFrame = tempGroupedDataFrame.reset_index()
tempGroupedDataFrame = tempGroupedDataFrame.set_index('year')
tempGroupedDataFrame = tempGroupedDataFrame.join(tempGroupedSeries, how='inner')
tempGroupedDataFrame = tempGroupedDataFrame.rename(columns={'numberPerMonth':'totalsPerYear'})
tempGroupedDataFrame['ratioPerYear'] = tempGroupedDataFrame['totalsPerYearPerProduct'] / tempGroupedDataFrame['totalsPerYear']
tempGroupedDataFrame = tempGroupedDataFrame.reset_index()
tempGroupedDataFrame['month'] = np.where(tempGroupedDataFrame['year'] < datetime.now().year, 12, datetime.now().month)
tempGroupedDataFrame['month_year'] = tempGroupedDataFrame['month'].apply(lambda x:str(x)) + '-' + tempGroupedDataFrame['year'].apply(lambda x:str(x))
tempGroupedDataFrame['month_year'] = pd.to_datetime(tempGroupedDataFrame['month_year'], format='%m-%Y', errors='coerce')

productTypePerYear = tempGroupedDataFrame

## Ratio of product per month. --IMPORTANT: Only the month_year column is set as timestamp format.--
tempGroupedSeries = rawTable.groupby(['year', 'month']).sum()['numberPerMonth']
tempGroupedDataFrame = rawTable
tempGroupedDataFrame = tempGroupedDataFrame.rename(columns={'numberPerMonth':'totalsPerMonthPerProduct'})
tempGroupedDataFrame = tempGroupedDataFrame.set_index(['year', 'month'])
tempGroupedDataFrame = tempGroupedDataFrame.join(tempGroupedSeries, how='inner')
tempGroupedDataFrame = tempGroupedDataFrame.rename(columns={'numberPerMonth':'totalsPerMonth'})
tempGroupedDataFrame['ratioPerMonth'] = tempGroupedDataFrame['totalsPerMonthPerProduct'] / tempGroupedDataFrame['totalsPerMonth']
tempGroupedDataFrame = tempGroupedDataFrame.reset_index()
tempGroupedDataFrame['month_year'] = tempGroupedDataFrame['month'].apply(lambda x:str(x)) + '-' + tempGroupedDataFrame['year'].apply(lambda x:str(x))
tempGroupedDataFrame['month_year'] = pd.to_datetime(tempGroupedDataFrame['month_year'], format='%m-%Y', errors='coerce')

productTypePerMonth = tempGroupedDataFrame

In [33]:
## Properties for graphs.

## Ratio of products per year.

## Colour.
productTypePerYear.loc[productTypePerYear[productTypePerYear['ratioPerYear'] >= 0.2].index,'colourInformation'] = 'green'
productTypePerYear.loc[productTypePerYear[(productTypePerYear['ratioPerYear'] >= 0.05) & (productTypePerYear['ratioPerYear'] < 0.2)].index,'colourInformation'] = 'orange'
productTypePerYear.loc[productTypePerYear[productTypePerYear['ratioPerYear'] < 0.05].index,'colourInformation'] = 'grey'
## Alpha.
productTypePerYear['alphaInformation'] = np.where(productTypePerYear['ratioPerYear'] >= 0.05, 0.9, 0.25)
## Legend.
productTypePerYear.loc[productTypePerYear[productTypePerYear['ratioPerYear'] >= 0.2].index,'legendInformation'] = 'High'
productTypePerYear.loc[productTypePerYear[(productTypePerYear['ratioPerYear'] >= 0.05) & (productTypePerYear['ratioPerYear'] < 0.2)].index,'legendInformation'] = 'Medium'
productTypePerYear.loc[productTypePerYear[productTypePerYear['ratioPerYear'] < 0.05].index,'legendInformation'] = 'Low'
## Tooltip values.
productTypePerYear['percentageValues'] = productTypePerYear['ratioPerYear'].apply(lambda x:str(round(x * 100, 1)))
productTypePerYear['dateValues'] = productTypePerYear['month_year'].apply(lambda x:x.strftime('%Y'))

## Ratio of products per month for 4 most interesting products.
productTypePerMonthRelevantProducts = productTypePerMonth[(productTypePerMonth['productType'] == 'mobile phone') |
                                                         (productTypePerMonth['productType'] == 'laptop') |
                                                         (productTypePerMonth['productType'] == 'hard drive') |
                                                         (productTypePerMonth['productType'] == 'base station')]
productTypePerMonthRelevantProducts = productTypePerMonthRelevantProducts.reset_index(drop=True)
## Line type.
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'mobile phone'].index,'lineTypeInformation'] = "solid"
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'laptop'].index,'lineTypeInformation'] = "dashed"
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'hard drive'].index,'lineTypeInformation'] = "dotted"
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'base station'].index,'lineTypeInformation'] = "dotdash"
# ## Colour.
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'mobile phone'].index,'colourInformation'] = "green"
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'laptop'].index,'colourInformation'] = "red"
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'hard drive'].index,'colourInformation'] = "orange"
productTypePerMonthRelevantProducts.loc[productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'base station'].index,'colourInformation'] = "black"
## Tooltip values.
productTypePerMonthRelevantProducts['percentageValues'] = productTypePerMonthRelevantProducts['ratioPerMonth'].apply(lambda x:str(round(x * 100, 1)))
productTypePerMonthRelevantProducts['dateValues'] = productTypePerMonthRelevantProducts['month_year'].apply(lambda x:x.strftime('%B-%Y'))

In [34]:
## Bokeh plot with product mix and time information.
output_notebook()

## Ratio of products per year.
sourceYear = ColumnDataSource(data=dict(x = productTypePerYear['month_year'],
                                         y = productTypePerYear['ratioPerYear'],
                                         fill_color = productTypePerYear['colourInformation'],
                                         fill_alpha = productTypePerYear['alphaInformation'],
                                         productType = productTypePerYear['productType'],
                                         percentageYear = productTypePerYear['percentageValues'],
                                         date = productTypePerYear['dateValues'],
                                         legend = productTypePerYear['legendInformation']))

hoverYear = HoverTool(tooltips=[
    ("Product", "@productType"),
    ("Year", "@date"),
    ("%Year", "@percentageYear")
])

fig1 = figure(plot_height=600, plot_width=700, title="Product mix per year", tools='reset, box_select, save, box_zoom, pan', x_axis_type="datetime",
             x_axis_label = "Date", y_axis_label = "Percentage contribution")
fig1.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
fig1.xaxis.axis_label_text_font_size = '20pt'
fig1.yaxis.axis_label_text_font_size = '20pt'
fig1.title.align = 'center'
fig1.title.text_font_size = '16pt'
fig1.xaxis.major_label_text_font_size = '12pt'
fig1.yaxis.major_label_text_font_size = '12pt'
glyphYear = fig1.circle(source = sourceYear,x='x', y='y', fill_color='fill_color',
                      size = 10,fill_alpha='fill_alpha', line_color=None, legend = "legend")
fig1.add_tools(hoverYear)

Loading BokehJS ...

In [35]:
show(fig1)

The graph above shows the product mix per year, the percentage contribution per year has been defined as:

$$\% = \frac{\sum_{j=1}^{n=12}Product_{ij}}{\sum_{p=1}^{m=12}Products}$$

where:

1. *P<sub>ij</sub>*: Products of type **i** in month **j**.
2. *Products*: Number of products of all types for month **p**. 

To focus on only the most relevant products, the percentage contributions has been coded as follows:

1. High: For products with more than 20% contribution.
2. Medium: For products with contributions between 5% and 20%.
3. Low: For products with contributions lower than 5%.

Only a single set of products falls within the High contribution category, other 4 fall within the Medium category while all others fall within the Low category.

1. High: Comprised by mobile phones.
2. Medium: Comprised by hard drives, laptops and base stations.


In [36]:
# Ratio of products per month.
sourceMonthPoints = ColumnDataSource(data=dict(x = productTypePerMonthRelevantProducts['month_year'],
                                         y = productTypePerMonthRelevantProducts['ratioPerMonth'],
                                         fill_color = productTypePerMonthRelevantProducts['colourInformation'],
                                         productType = productTypePerMonthRelevantProducts['productType'],
                                         percentageMonth = productTypePerMonthRelevantProducts['percentageValues'],
                                         date = productTypePerMonthRelevantProducts['dateValues']))

hoverMonth = HoverTool(tooltips=[
    ("Product", "@productType"),
    ("Year", "@date"),
    ("%Year", "@percentageMonth")
])

## Dataframe for mobile phones.
phonesPerMonth = productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'mobile phone']
sourceMonthPhone = ColumnDataSource(data=dict(x = phonesPerMonth['month_year'],
                                         y = phonesPerMonth['ratioPerMonth'],
                                         productType = phonesPerMonth['productType'],
                                         percentageMonth =  phonesPerMonth['percentageValues'],
                                         date = phonesPerMonth['dateValues']))
## Dataframe for hard drives.
hardDrivesPerMonth = productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'hard drive']
sourceMonthHardDrives = ColumnDataSource(data=dict(x = hardDrivesPerMonth['month_year'],
                                         y = hardDrivesPerMonth['ratioPerMonth'],
                                         productType = hardDrivesPerMonth['productType'],
                                         percentageMonth =  hardDrivesPerMonth['percentageValues'],
                                         date = hardDrivesPerMonth['dateValues']))
## Dataframe for laptops.
laptopsPerMonth = productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'laptop']
sourceMonthLaptop = ColumnDataSource(data=dict(x = laptopsPerMonth['month_year'],
                                         y = laptopsPerMonth['ratioPerMonth'],
                                         productType = laptopsPerMonth['productType'],
                                         percentageMonth =  laptopsPerMonth['percentageValues'],
                                         date = laptopsPerMonth['dateValues']))
## Dataframe for base stations.
baseStationsPerMonth = productTypePerMonthRelevantProducts[productTypePerMonthRelevantProducts['productType'] == 'base station']
sourceMonthBaseStation = ColumnDataSource(data=dict(x = baseStationsPerMonth['month_year'],
                                         y = baseStationsPerMonth['ratioPerMonth'],
                                         productType = baseStationsPerMonth['productType'],
                                         percentageMonth =  baseStationsPerMonth['percentageValues'],
                                         date = baseStationsPerMonth['dateValues']))

fig2 = figure(plot_height=600, plot_width=700, title="Product mix per month for highest contributing products", tools='reset, box_select, save, box_zoom, pan', x_axis_type="datetime",
             x_axis_label = "Date", y_axis_label = "Percentage contribution")
fig2.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
fig2.xaxis.axis_label_text_font_size = '20pt'
fig2.yaxis.axis_label_text_font_size = '20pt'
fig2.title.align = 'center'
fig2.title.text_font_size = '16pt'
fig2.xaxis.major_label_text_font_size = '12pt'
fig2.yaxis.major_label_text_font_size = '12pt'
glyphMonthPoints = fig2.circle(source = sourceMonthPoints, x = 'x', y = 'y', fill_color='fill_color', fill_alpha = 0.9, line_color=None, legend = 'productType', size = 10)
glyphMonthPhones = fig2.line(source = sourceMonthPhone, x = 'x', y = 'y', line_dash = 'solid', line_color = 'green')
glyphHardDrives = fig2.line(source = sourceMonthHardDrives, x = 'x', y = 'y', line_dash = 'dotted', line_color = 'orange')
glyphLaptop = fig2.line(source = sourceMonthLaptop, x = 'x', y = 'y', line_dash = 'dashed', line_color = 'red')
glyphBaseStation = fig2.line(source = sourceMonthBaseStation, x = 'x', y = 'y', line_dash = 'dotdash', line_color = 'black')
fig2.add_tools(hoverMonth)

In [37]:
show(fig2)

Looking at the product mix composition per month it is clear that mobile phones are the major contributor in terms of product mix, posterior analysis will focus on mobile phones. From the graph above the following interesting behaviours are found:

1. The mobile phone contribution has been declining through the whole 20 months, it started as 97% and has fallen to 55%. There have been some periods where the contribution has increased for example from August 2016 to December 2016 but in general the trend is a decreasing contribution.
2. Although the contribution of the other 3 major products (base station, laptop, hard drives) has been increasing slowly in the past 20 months their contribution falls below the contribution of mobiles phones by a considerable margin. For June 2017 the difference in contribution between mobile phones is 40%, 50% and 53.7% and for hard drives, base stations and laptops respectively.
3. For the major products there are periods where the product mix contribution increases considerably within a span of 1 to 4 months. For example, the mobile phone contribution jumped from 57.2% to 85.1% from January 2017 to February 2017 and the base station contribution jumped from 5.8% to 21.5% from April 2017 to May 2017. It is possible that the increase is due to a customer renewing their equipment therefore recycling a large number of old products, since information on which customer the products are coming from is recorded this can be analysed to learn when major customers are expected to renew their equipment.

## Product comparison web scrapping

As mentioned in the problem overview Company "X" buys equipment from other companies and individual users to recover as much as possible from the product. Several other companies like Mazuma Mobile in UK or SmartPhone Trade-In in USA offer the same service. Since the highest contribution to the product mix is of mobile phones this part of the analysis will be focused on mobile phone only.

Company "X" makes an offer in for the phone upfront based on 2 methods:

1. For individual users they will base the offer on physical attributes input by the users themselves. This attributes include if the phone is working, if it is scratched and others. Based on the attributes ranking values the offer will be lowered.
2. For companies who sell items in bulk the offer will be made after Company "X" has physically reviewed the items themselves.

Notice that the actual payment for the phone occurs only after the phones have been physically reviewed and it is Company "X" who pays for picking up the products and delivering them back if they are rejected. Furthermore Company "X" must make an offer at least as good as the one made by other recycling companies otherwise customers will take their business somewhere else.

Before making an offer an employee will go to a comparison website (e.g. sellmymobile.com) and check what other companies are offering. Company "X" would benefit from automating this process as well as learning about the historical price for that phone as that would give them an indication of how much profit they can expect to make. If the phone price depreciation is high enough you could end up buying a phone whose final sell point will be lower than processing costs. This is even more relevant as the phones bought in the best of cases will be in "As new" conditions.

To prove the how the automation of the process would work I will use the IPhone 5s with 16GB memory. I will web scrape the information from several price comparison websites (4 in total) and present the results in a graph. The condition of the phone will be set as working for this example.

The historical prices will be webscraped from pricespy.co.uk. and camelcamelcamel.com

Because one of the websites displays the price in dollars and Company "X" is based in UK a conversion rate retrieved from ExchangeRate-API will be used.


In [38]:
def parseSellMyMobile(urlAddress):
    """
    This function parses the input from the sellmymobile
    price comparison web page. The url address must point
    to the actual results web page. Dealing with the selection
    of the product is done in different function.

    Parameters
    ----------
    urlAddress : string
        The url address from which the result data will be parsed from.

    Returns
    -------
    resultsList : List 
        List of tuples where the first member of the tuple is the
        buyer and the second member of the tuple is the price offered.
    """
    ## Step 1. For each result parse the buyer and the price offered.
    page = requests.get(urlAddress)
    soup = BeautifulSoup(page.text, "lxml")
    ## Step 1.1. Get the buyer name.
    namesList = []
    tempList = soup.select('td.merchantlogo')
    strFilter = r"m2:'([^,]+)'{1}"
    for tag in tempList:
        tempString = tag.select('a')[0]["onclick"]
        tempString = re.findall(strFilter, tempString)[0]
        namesList.append(tempString)
    ## Step 1.2. Get the price.
    pricesList = []
    tempList = soup.select('td.price')
    strFilter = r'£(.*)'
    for tag in tempList:
        tempString = tag.text.strip()
        tempString = re.findall(strFilter, tempString)[0]
        pricesList.append(tempString)
    pricesList = map(lambda x:round(float(x), 1), pricesList)
    ## Step 2. Return a list with the results.
    resultsList = list(zip(namesList, pricesList))
    resultsList = [(x[0],x[1], "sellmymobile") for x in resultsList]
    return resultsList

In [39]:
def parseCompareMyMobile(urlAddress):
    """
    This function parses the input from the compareMyMobile
    price comparison web page. The url address must point
    to the actual results web page. Dealing with the selection
    of the product is done in different function.

    Parameters
    ----------
    urlAddress : string
        The url address from which the result data will be parsed from.

    Returns
    -------
    resultsList : List
        List of tuples where the first member of the tuple is the
        buyer and the second member of the tuple is the price offered.
    """
    ## Step 1. For each result parse the buyer and the price offered.
    page = requests.get(urlAddress)
    soup = BeautifulSoup(page.text, "lxml")
    ## Step 1.1. Get the buyer name.
    namesList = []
    tempList = soup.select('div.sell-col-1')
    tempList = tempList[1:]
    strFilter = r"'(.*)'{1},"
    for tag in tempList:
        tempString = tag.select('a')[0]["onclick"]
        tempString = re.findall(strFilter, tempString)[0]
        namesList.append(tempString)
    ## Step 1.2. Get the price.
    pricesList = []
    tempList = soup.select('div.sell-col-2 > span.filter-price')
    strFilter = r'£(.*)'
    for tag in tempList:
        tempString = tag.text.strip()
        tempString = re.findall(strFilter, tempString)[0]
        pricesList.append(tempString)
    pricesList = map(lambda x:round(float(x), 1), pricesList)
    ## Step 2. Return a list with the results.
    resultsList = list(zip(namesList, pricesList))
    resultsList = [(x[0],x[1], "comparemymobile") for x in resultsList]
    return resultsList

In [40]:
def parseMobileValuer(urlAddress):
    """
    This function parses the input from the mobileValuer
    price comparison web page. The url address must point
    to the actual results web page. Dealing with the selection
    of the product is done in different function.

    Parameters
    ----------
    urlAddress : string
        The url address from which the result data will be parsed from.

    Returns
    -------
    resultsList : List
        List of tuples where the first member of the tuple is the
        buyer and the second member of the tuple is the price offered.
    """
    ## Step 1. For each result parse the buyer and the price offered.
    page = requests.get(urlAddress)
    soup = BeautifulSoup(page.text, "lxml")
    ## Step 1.1. Get the buyer name.
    namesList = []
    tempList = soup.select('a.retailerlink')
    strFilter = r'^[A-Z][^\*]+'
    for tag in tempList:
        tempString = tag.text.strip()
        tempString = re.findall(strFilter, tempString)[0]
        tempString = tempString.strip()
        namesList.append(tempString)
    ## Step 1.2. Get the price.
    pricesList = []
    tempList = soup.select('td.sortedcol')
    strFilter = r'£(.*)'
    for tag in tempList:
        tempString = tag.text.strip()
        tempString = re.findall(strFilter, tempString)[0]
        pricesList.append(tempString)
    pricesList = map(lambda x:round(float(x), 1), pricesList)
    ## Step 2. Return a list with the results.
    resultsList = list(zip(namesList, pricesList))
    resultsList = [(x[0],x[1], "mobilevaluer") for x in resultsList]
    return resultsList

In [41]:
def parseSmartPhoneTradeIn(urlAddress, exchangeRate):
    """
    This function parses the input from the smartphonetradein
    price comparison web page. The url address must point
    to the actual results web page. Dealing with the selection
    of the product is done in different function.

    Parameters
    ----------
    urlAddress : string
        The url address from which the result data will be parsed from.
    exchangerate : float
        The exchange rate to be used to convert from $US to £UK.

    Returns
    -------
    resultsList : List
        List of tuples where the first member of the tuple is the
        buyer and the second member of the tuple is the price offered.
    """
    ## Step 1. For each result parse the buyer and the price offered.
    page = requests.get(urlAddress)
    soup = BeautifulSoup(page.text, "lxml")
    ## Step 1.1. Get the buyer name.
    namesList = []
    tempList = soup.select('div#list_view_product_name')
    for tag in tempList:
        tempString = tag.text.strip()
        namesList.append(tempString)
    ## Step 1.2. Get the price.
    pricesList = []
    tempList = soup.select('div#list_view_product_price')
    strFilter = r'\$(.*)'
    for tag in tempList:
        tempString = tag.text.strip()
        tempString = re.findall(strFilter, tempString)[0]
        pricesList.append(tempString)
    pricesList = map(lambda x:float(x), pricesList)
    pricesList = map(lambda x:round(x * exchangeRate, 1), pricesList)
    ## Step 2. Return a list with the results.
    resultsList = list(zip(namesList, pricesList))
    resultsList = [(x[0],x[1], "smartphonetradein") for x in resultsList]
    return resultsList

In [42]:
def exchangeRateUSToUK(exchangeRateAPICredentials):
    """
    This function retrieves the exchange rate for $US to £. It
    retrieves the information using ExchangeRate-API.
    
    Parameters
    ----------
    exchangeRateAPICredentials : string
        API credentials.
    
    Returns
    -------
    exchangeRate : float
        Exchange rate from $US to £UK.
    """
    urlbase = 'https://v3.exchangerate-api.com/bulk'
    url = urlbase + '/' + exchangeRateAPICredentials + '/USD' 
    response = requests.get(url)
    data = response.json()
    exchangeRate = data['rates']['GBP']
    return exchangeRate

In [43]:
## Example web scrapping price comparison site.
import requests
from bs4 import BeautifulSoup

## Step 0. Set the url for the different web pages to scrape from.
urlBases = ['http://www.sellmymobile.com/phone/apple-iphone-5s-16gb',
            'https://www.comparemymobile.com/recycle/apple/iphone-5s-16gb',
            'http://mobilevaluer.com/mobile/working/apple/iphone-5s-16gb/',
            'https://smartphonetradein.com/search-results-page/?dp_s=Iphone+5s+']
## Step 2. Request and parse the price data from each site. A pandas dataframe containing the companies name and
## the offered price will be returned.
exchangeRate = 0.78107479 #exchangeRateUSToUK(exchangeRateAPICredentials)

sellMyMobile = parseSellMyMobile(urlBases[0])
compareMyMobile = parseCompareMyMobile(urlBases[1])
mobileValuer = parseMobileValuer(urlBases[2])
smartPhoneTradeIn = parseSmartPhoneTradeIn(urlBases[3], exchangeRate)

resultsList = [sellMyMobile, compareMyMobile,
               mobileValuer, smartPhoneTradeIn]

## Step 2.1. Create the dataframe for analysis.
buyer = []
price = []
source = []
for pageResult in resultsList:
    for result in pageResult:
        buyer.append(result[0])
        price.append(result[1])
        source.append(result[2])
resultsDataFrame = pd.DataFrame({
    'buyer':buyer,
    'price':price,
    'source':source
})


In [44]:
resultsDataFrameGrouped = resultsDataFrame.groupby(['source'])

In [45]:
resultsDataFrameGrouped.describe()

price                                                      
                  count       mean        std   min     25%   50%   75%   max
source                                                                       
comparemymobile    10.0  42.100000  12.187881  20.0  32.500  45.0  50.0  56.0
sellmymobile       14.0  45.428571  12.840527  20.0  32.500  50.0  55.0  60.0
smartphonetradein   6.0  20.816667  14.129036   7.8   8.775  19.5  27.3  43.0

In [46]:
type(resultsDataFrameGrouped.describe())

pandas.core.frame.DataFrame

In [47]:
## Summary information for each source.
resultsDataFrameGrouped = resultsDataFrame.groupby(['source'])
summaryResultsMean = resultsDataFrameGrouped.mean()
summaryResultsMean = summaryResultsMean.reset_index()
summaryResultsMean['buyer'] = 'Mean value'
summaryResultsQ1 = resultsDataFrameGrouped.quantile(q = 0.25)
summaryResultsQ2 = resultsDataFrameGrouped.quantile(q = 0.5)
summaryResultsQ3 = resultsDataFrameGrouped.quantile(q = 0.75)
interQuantileRange = summaryResultsQ3 - summaryResultsQ1
upperLimit = summaryResultsQ3 + 1.5 * interQuantileRange
lowerLimit = summaryResultsQ1 - 1.5 * interQuantileRange

In [48]:
# Price per from each website.
sourcePrices = ColumnDataSource(data=dict(x = resultsDataFrame['source'],
                                          y = resultsDataFrame['price'],
                                          buyer = resultsDataFrame['buyer']))

sourceMeanPrices = ColumnDataSource(data=dict(x = summaryResultsMean['source'],
                                          y = summaryResultsMean['price'],
                                          buyer = summaryResultsMean['buyer']))

hoverPrices = HoverTool(tooltips=[
    ("Buyer", "@buyer"),
    ("Price (£)", "@y{1.1}")
])

fig3 = figure(plot_height=600, plot_width=800, title="Phone price comparison", tools='reset, box_select, save, box_zoom, pan',
             x_axis_label = "Source", y_axis_label = "Offered price (£)", x_range = [x for x in set(resultsDataFrame['source'])])
fig3.xaxis.axis_label_text_font_size = '20pt'
fig3.yaxis.axis_label_text_font_size = '20pt'
fig3.title.align = 'center'
fig3.title.text_font_size = '20pt'
fig3.xaxis.major_label_text_font_size = '12pt'
fig3.yaxis.major_label_text_font_size = '12pt'
fig3.yaxis[0].formatter = NumeralTickFormatter(format="£0.00")
fig3.circle(source = sourcePrices, x = 'x', y = 'y', fill_alpha = 0.9, line_color=None, size = 10)
fig3.square(source = sourceMeanPrices, x = 'x', y = 'y', fill_alpha = 0.9, line_color=None, size = 12, color = 'firebrick')
fig3.add_tools(hoverPrices)

In [49]:
show(fig3)